<a href="https://colab.research.google.com/github/elyager/LLMs-from-scratch/blob/main/LLM_from_scratch_personal_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Requiriments

In [1]:
!pip install tiktoken
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0+cu124


### Vocabulary

- **vocab_size:** es el tamaño del vocabulario en tokens únicos disponibles+extensiones. En nuestro caso dado por el tokenizador creado con BPE.
- **output_dim:** el número de dimensiones de cada token. Las dimensiones describen a una palabra o concepto. Más dimensiones capturan más detalles.
- **max_length:** es la máxima logitud de tokens por secuencia.
- **batch_size:** es cuantas secuencias (muestras de texto) tiene cada batch. Larger batch sizes can speed up training but might require more memory.
- **stride:** el tamaño de la zancada en tokens, cuantos tokens salta para la siguiente secuencia, esto determina que tanto se empalma una secuencia con otra. Ensure the model sees the context of each token multiple times during training, helping it learn better relationships between words.
- **shuffle:** determina si le da un orden aleatorio a las secuencias para que sea random en cada epoc. Prevents the model from learning patterns based on the order of data presentation. It helps generalize learning and avoid overfitting to a specific data order.
-**drop_last:** indica si se debe descartar el último batch cuando no cumple con el número de muestras establecido en max_lenght. Si se tiene un set de datos pequeño es mejor no hacer drop.
-**num_workers:** es el número de procesos, a mayor número más rápidez.
-**attention score:** determina qué tan "similar" es una pababra con la otra a través de dot product que es un tipo de similarity function. Mayor atention score mayor similitud entre los números.
--**attention weight:** es la versión normalizada a través de softmax de los attention scores.
-**context vector:** es un embedding vector pero que tiene todo el contexto del resto de input vectors. Se obtiene sumando todos los attention weights del inpute secuence.

# Chapter 1 & 2

### Load the text for training (our corpus)

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### BytePair Encoding (BPE)


In [3]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [4]:
tokenizer = tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text, allowed_special={"<|endoftext|>"})
enc_text.append(tokenizer.eot_token)

# First 10 tokens from raw_text
first_10_token_ids = enc_text[:10]
decoded_tokens = [tokenizer.decode([token_id]) for token_id in first_10_token_ids]
delimited_tokens = ' |-|'.join(decoded_tokens)
print(delimited_tokens)
print(enc_text[:10])
print(f'\n Total of tokens: {len(enc_text)}')

I |-| H |-|AD |-| always |-| thought |-| Jack |-| G |-|is |-|burn |-| rather
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138]

 Total of tokens: 5146


### Dataset loader (creating tokenIDs for inputs and targets)

In [5]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        token_ids.append(tokenizer.eot_token)

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

# dataset = GPTDatasetV1(raw_text, tokenizer, max_length=4, stride=1)
# print(dataset.input_ids)
# print(dataset.target_ids)

In [6]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

### Use DataLoader

In [7]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

# for batch in dataloader:
#     input, target = batch
#     print(input, target)

data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch) # input and target
second_batch = next(data_iter)
print(second_batch) # input and target

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [8]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
# First batch
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)
print("Inputs:\n", tokenizer.decode(inputs[0].tolist()))
print("\nTargets:\n", tokenizer.decode(targets[0].tolist()))

# Second batch
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)
print("Inputs:\n", tokenizer.decode(inputs[0].tolist()))
print("\nTargets:\n", tokenizer.decode(targets[0].tolist()))

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
Inputs:
 I HAD always

Targets:
  HAD always thought
Inputs:
 tensor([[  287,   262,  6001,   286],
        [  465, 13476,    11,   339],
        [  550,  5710,   465, 12036],
        [   11,  6405,   257,  5527],
        [27075,    11,   290,  4920],
        [ 2241,   287,   257,  4489],
        [   64,   319,   262, 34686],
        [41976,    13,   357, 10915]])

Ta

### Create our token embedding layer

In [9]:
vocab_size = 50257
output_dim = 3 # 256 is a more common starting point

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(token_embedding_layer.weight.shape)
print(token_embedding_layer.weight)

torch.Size([50257, 3])
Parameter containing:
tensor([[ 1.0499,  1.5613,  1.5356],
        [ 0.0054,  0.4545, -0.2530],
        [ 0.6044, -0.0970,  1.2713],
        ...,
        [ 0.8461,  0.2061, -0.7791],
        [ 0.1027,  0.4897,  0.3037],
        [ 1.8263,  1.7426,  0.0666]], requires_grad=True)


#### Load our dataset to get the inputs

In [10]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs:\n",  inputs) # we take the first batch and  ignore the targets for now
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


### Create token embeddings

In [11]:
token_embeddings = token_embedding_layer(inputs)
# each token now has the assigned number of dimentions instead of being a single token ID
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
print(token_embeddings)

torch.Size([8, 4, 3])
tensor([[[-1.2239,  0.9454, -1.0618],
         [ 1.6218, -0.3814,  0.6126],
         [ 0.1450,  1.1302, -0.6425],
         [-0.5892, -1.5937,  0.0803]],

        [[-0.8558, -0.9859,  0.2085],
         [ 0.3994, -0.3635,  0.2396],
         [ 0.4786,  0.4042, -1.1887],
         [ 0.6692,  0.0512,  1.8413]],

        [[-0.2117, -2.3216, -1.0187],
         [ 0.1332,  1.5330,  0.2707],
         [-0.2937, -0.0921, -1.4307],
         [ 0.3152, -0.6272, -1.1681]],

        [[-0.3351, -1.3230, -1.5735],
         [-0.9770, -0.3570,  0.9478],
         [ 0.3268, -1.7190,  0.0553],
         [-0.2937, -0.0921, -1.4307]],

        [[ 0.8881,  0.8748, -0.4730],
         [ 1.9018,  0.9325,  0.1734],
         [ 0.4939,  0.5552, -0.2040],
         [-0.9770, -0.3570,  0.9478]],

        [[-1.1258,  1.8030, -0.1891],
         [ 1.3732,  0.1376, -0.0471],
         [-0.9402,  0.0433,  1.1018],
         [-0.3877,  0.1870,  0.2711]],

        [[ 0.0903,  2.2378,  2.1044],
         [-0.616

### Create absolute positional embeddings

In [12]:
pos_embedding_layer = torch.nn.Embedding(max_length, output_dim)

# # [0, 1, 2, 3] "column" position is the position of each word on each sequence of 4 context_length
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# # uncomment & execute the following line to see how the embeddings look like
print(pos_embeddings)

torch.Size([4, 3])
tensor([[-0.2958,  0.0469, -0.1967],
        [ 0.7208,  0.6753, -0.3788],
        [-0.5114,  0.2414,  1.1345],
        [-0.6289,  0.0640,  0.8413]], grad_fn=<EmbeddingBackward0>)


### Create input embeddings

In [13]:
input_embeddings = token_embeddings + pos_embeddings
print(f'{token_embeddings.shape} + {pos_embeddings.shape} = {input_embeddings.shape}')

# uncomment & execute the following line to see how the embeddings look like
print(input_embeddings)

torch.Size([8, 4, 3]) + torch.Size([4, 3]) = torch.Size([8, 4, 3])
tensor([[[-1.5197,  0.9923, -1.2585],
         [ 2.3426,  0.2939,  0.2338],
         [-0.3665,  1.3716,  0.4920],
         [-1.2181, -1.5297,  0.9216]],

        [[-1.1515, -0.9390,  0.0118],
         [ 1.1202,  0.3118, -0.1392],
         [-0.0329,  0.6456, -0.0542],
         [ 0.0402,  0.1152,  2.6826]],

        [[-0.5075, -2.2746, -1.2154],
         [ 0.8540,  2.2083, -0.1081],
         [-0.8052,  0.1493, -0.2962],
         [-0.3137, -0.5632, -0.3269]],

        [[-0.6309, -1.2760, -1.7702],
         [-0.2562,  0.3183,  0.5690],
         [-0.1846, -1.4776,  1.1898],
         [-0.9227, -0.0280, -0.5894]],

        [[ 0.5923,  0.9218, -0.6697],
         [ 2.6226,  1.6078, -0.2054],
         [-0.0176,  0.7966,  0.9305],
         [-1.6060, -0.2930,  1.7891]],

        [[-1.4216,  1.8499, -0.3858],
         [ 2.0940,  0.8129, -0.4260],
         [-1.4516,  0.2847,  2.2363],
         [-1.0166,  0.2510,  1.1124]],

        [

### What about more dimmension?

In [14]:
def more_more_more_dimensions(dim):
  vocab_size = 50257
  output_dim = dim

  token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
  token_embeddings = token_embedding_layer(inputs)
  print(token_embeddings.shape)
  print(token_embeddings)

  pos_embedding_layer = torch.nn.Embedding(max_length, output_dim)
  pos_embeddings = pos_embedding_layer(torch.arange(max_length))
  print(pos_embeddings.shape)
  print(pos_embeddings)


  input_embeddings = token_embeddings + pos_embeddings
  print(f'{token_embeddings.shape} + {pos_embeddings.shape} = {input_embeddings.shape}')
  print(input_embeddings)

more_more_more_dimensions(256)  #256 is a more common starting point

torch.Size([8, 4, 256])
tensor([[[ 0.4284, -0.0444, -1.5124,  ..., -1.1995,  1.2147,  0.1603],
         [ 0.9657,  0.6022,  1.4287,  ..., -1.0535, -1.4809,  0.7132],
         [-0.4677, -1.9291, -0.1119,  ..., -0.5154,  0.3336,  0.1206],
         [-0.1478, -0.9211,  0.2747,  ...,  1.8229,  0.4754, -0.9500]],

        [[-2.0476, -1.9832, -1.4409,  ..., -0.6543, -0.3288, -0.4750],
         [ 0.3891,  0.2687,  0.0831,  ...,  0.3362,  1.1893,  0.4639],
         [-0.3365,  0.6894,  0.1393,  ..., -0.9712, -1.0528,  2.0448],
         [ 0.0072, -0.4715, -0.5916,  ...,  0.5685,  0.0201, -0.8140]],

        [[ 0.8937, -1.0620, -1.6252,  ..., -0.1977, -1.7489,  0.0119],
         [-1.1047, -0.3365,  1.0697,  ...,  1.5743, -2.3146,  1.9848],
         [-0.8749, -1.7211, -1.0064,  ...,  0.1872, -0.8916,  0.9201],
         [-0.2906, -1.0940, -0.5154,  ..., -0.0558, -1.2010,  1.0564]],

        ...,

        [[ 0.6405,  0.6699,  1.3605,  ...,  1.1404,  0.4990,  1.4791],
         [ 1.2230, -1.5099,  1.26

# Chapter 3

## Preparing data to work with

### Get input embeddings

In [15]:
def get_input_embeddings(my_raw_text):
  dataloader = create_dataloader_v1(my_raw_text, batch_size=1, max_length=6, stride=6, shuffle=False)

  data_iter = iter(dataloader)
  # First and only batch
  inputs, targets = next(data_iter)
  # print(inputs)
  # print(targets)  # ignore the targets

  vocab_size = 50257
  output_dim = 3

  token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
  token_embeddings = token_embedding_layer(inputs)
  context_length = 6
  pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
  pos_embeddings = pos_embedding_layer(torch.arange(context_length))

  input_embeddings = token_embeddings + pos_embeddings
  # print(f'{token_embeddings.shape} + {pos_embeddings.shape} = {input_embeddings.shape}')
  # print(input_embeddings)
  return input_embeddings

In [16]:
my_raw_text = "Your journey starts with one step"
small_input_embeddings = get_input_embeddings(my_raw_text)[0]
print(small_input_embeddings)

tensor([[-1.0610, -0.3015, -1.7378],
        [ 0.8547, -0.7702, -0.6719],
        [ 2.6666,  2.5809, -0.0577],
        [-0.2783, -0.9558, -0.0836],
        [-0.3893, -0.3262, -2.2585],
        [ 0.2071,  2.5861,  1.3067]], grad_fn=<SelectBackward0>)


In [17]:
# tensor([
#     [ 0.4113,  1.3397, -1.2234], Your    (x^1)
#     [-1.8881, -0.0679, -1.1267], journey (x^2)
#     [-0.2323, -2.2089, -1.6685], starts  (x^3)
#     [ 0.5615,  1.2698,  2.5768], with    (x^4)
#     [-0.9290, -0.0227,  0.6467], one     (x^5)
#     [ 0.5691, -2.0627, -3.2411]  step    (x^6)
# ])

### Forcing values to fit between 0 and 1

In [18]:
import decimal
min_val = small_input_embeddings.min()
max_val = small_input_embeddings.max()
scaled_embeddings = (small_input_embeddings - min_val) / (max_val - min_val)
rounded_embeddings = torch.round(scaled_embeddings * 100) / 100
print(rounded_embeddings)

tensor([[0.2400, 0.4000, 0.1100],
        [0.6300, 0.3000, 0.3200],
        [1.0000, 0.9800, 0.4500],
        [0.4000, 0.2600, 0.4400],
        [0.3800, 0.3900, 0.0000],
        [0.5000, 0.9800, 0.7200]], grad_fn=<DivBackward0>)


## Simple self-attention

### Step 1 - Compute unormalized attention scores

In [19]:
query = rounded_embeddings[1]  # 2nd input token is the query)
# just allocate a tensor in memory with 6 spaces
attn_scores_2 = torch.empty(rounded_embeddings.shape[0])

#fill the tensor with the dot products which multiply and sum
for i, x_i in enumerate(rounded_embeddings):
    print(f'dot product of {x_i} against journey {query}')
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

dot product of tensor([0.2400, 0.4000, 0.1100], grad_fn=<UnbindBackward0>) against journey tensor([0.6300, 0.3000, 0.3200], grad_fn=<SelectBackward0>)
dot product of tensor([0.6300, 0.3000, 0.3200], grad_fn=<UnbindBackward0>) against journey tensor([0.6300, 0.3000, 0.3200], grad_fn=<SelectBackward0>)
dot product of tensor([1.0000, 0.9800, 0.4500], grad_fn=<UnbindBackward0>) against journey tensor([0.6300, 0.3000, 0.3200], grad_fn=<SelectBackward0>)
dot product of tensor([0.4000, 0.2600, 0.4400], grad_fn=<UnbindBackward0>) against journey tensor([0.6300, 0.3000, 0.3200], grad_fn=<SelectBackward0>)
dot product of tensor([0.3800, 0.3900, 0.0000], grad_fn=<UnbindBackward0>) against journey tensor([0.6300, 0.3000, 0.3200], grad_fn=<SelectBackward0>)
dot product of tensor([0.5000, 0.9800, 0.7200], grad_fn=<UnbindBackward0>) against journey tensor([0.6300, 0.3000, 0.3200], grad_fn=<SelectBackward0>)
tensor([0.3064, 0.5893, 1.0680, 0.4708, 0.3564, 0.8394], grad_fn=<CopySlices>)


### Step 2 - Normalize the attenton scores to sum up to 1

In [20]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1190, 0.1579, 0.2549, 0.1403, 0.1251, 0.2028],
       grad_fn=<DivBackward0>)
Sum: tensor(1.0000, grad_fn=<SumBackward0>)


In [21]:
# Fooling around with dimensions on vectors
my_tensor = torch.ones(2,3,4) # I always start with the last dimension which is [-1]
print(my_tensor)
print(my_tensor.shape)
print(my_tensor.shape[-1]) # last dimension
print(my_tensor.shape[0]) # first dimenson
print(my_tensor.shape[1])
print(my_tensor.shape[2]) # same as [-1]
# print(my_tensor.shape[3]) # IndexError

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])
torch.Size([2, 3, 4])
4
2
3
4


In [22]:
# using pytorch softmax fucntion
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum()) #100%


Attention weights: tensor([0.1190, 0.1579, 0.2549, 0.1403, 0.1251, 0.2028],
       grad_fn=<SoftmaxBackward0>)
Sum: tensor(1.0000, grad_fn=<SumBackward0>)


### Step 3 - Compute the context vector $z^{(2)}$

In [23]:
query = rounded_embeddings[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
attn_weights_sum = 0
for i,x_i in enumerate(rounded_embeddings):
    print(f'{attn_weights_2[i]} * {x_i}')
    context_vec_2 += attn_weights_2[i]*x_i
    attn_weights_sum += attn_weights_2[i]

print(attn_weights_sum)
print(context_vec_2)

0.11900985240936279 * tensor([0.2400, 0.4000, 0.1100], grad_fn=<UnbindBackward0>)
0.15792278945446014 * tensor([0.6300, 0.3000, 0.3200], grad_fn=<UnbindBackward0>)
0.25488343834877014 * tensor([1.0000, 0.9800, 0.4500], grad_fn=<UnbindBackward0>)
0.14027521014213562 * tensor([0.4000, 0.2600, 0.4400], grad_fn=<UnbindBackward0>)
0.1251116245985031 * tensor([0.3800, 0.3900, 0.0000], grad_fn=<UnbindBackward0>)
0.20279717445373535 * tensor([0.5000, 0.9800, 0.7200], grad_fn=<UnbindBackward0>)
tensor(1.0000, grad_fn=<AddBackward0>)
tensor([0.5880, 0.6288, 0.3861], grad_fn=<AddBackward0>)


### Get All attention weights

In [24]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(rounded_embeddings):
    for j, x_j in enumerate(rounded_embeddings):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.2297, 0.3064, 0.6815, 0.2484, 0.2472, 0.5912],
        [0.3064, 0.5893, 1.0680, 0.4708, 0.3564, 0.8394],
        [0.6815, 1.0680, 2.1629, 0.8528, 0.7622, 1.7844],
        [0.2484, 0.4708, 0.8528, 0.4212, 0.2534, 0.7716],
        [0.2472, 0.3564, 0.7622, 0.2534, 0.2965, 0.5722],
        [0.5912, 0.8394, 1.7844, 0.7716, 0.5722, 1.7288]],
       grad_fn=<CopySlices>)


We can achive the same but more efficiently via matrix multiplication

In [25]:
# Using matrix transpose (remember is row * column)
print(rounded_embeddings.shape)
print(rounded_embeddings.T.shape)
print('-------------------')
print(rounded_embeddings)
print('-------------------')
print(rounded_embeddings.T)

torch.Size([6, 3])
torch.Size([3, 6])
-------------------
tensor([[0.2400, 0.4000, 0.1100],
        [0.6300, 0.3000, 0.3200],
        [1.0000, 0.9800, 0.4500],
        [0.4000, 0.2600, 0.4400],
        [0.3800, 0.3900, 0.0000],
        [0.5000, 0.9800, 0.7200]], grad_fn=<DivBackward0>)
-------------------
tensor([[0.2400, 0.6300, 1.0000, 0.4000, 0.3800, 0.5000],
        [0.4000, 0.3000, 0.9800, 0.2600, 0.3900, 0.9800],
        [0.1100, 0.3200, 0.4500, 0.4400, 0.0000, 0.7200]],
       grad_fn=<PermuteBackward0>)


In [26]:
attn_scores = rounded_embeddings @ rounded_embeddings.T
print(attn_scores)

tensor([[0.2297, 0.3064, 0.6815, 0.2484, 0.2472, 0.5912],
        [0.3064, 0.5893, 1.0680, 0.4708, 0.3564, 0.8394],
        [0.6815, 1.0680, 2.1629, 0.8528, 0.7622, 1.7844],
        [0.2484, 0.4708, 0.8528, 0.4212, 0.2534, 0.7716],
        [0.2472, 0.3564, 0.7622, 0.2534, 0.2965, 0.5722],
        [0.5912, 0.8394, 1.7844, 0.7716, 0.5722, 1.7288]],
       grad_fn=<MmBackward0>)


Apply softmax

In [27]:
# dim=-1 is to apply softmax to the last dimenison, in this case rows
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.1404, 0.1516, 0.2206, 0.1430, 0.1429, 0.2015],
        [0.1190, 0.1579, 0.2549, 0.1403, 0.1251, 0.2028],
        [0.0823, 0.1211, 0.3619, 0.0976, 0.0892, 0.2479],
        [0.1256, 0.1569, 0.2299, 0.1493, 0.1263, 0.2120],
        [0.1383, 0.1543, 0.2315, 0.1392, 0.1453, 0.1914],
        [0.0919, 0.1178, 0.3032, 0.1101, 0.0902, 0.2868]],
       grad_fn=<SoftmaxBackward0>)


In [28]:
row_0_sum = sum([0.1403, 0.1365, 0.1915, 0.1552, 0.1659, 0.2106])
print(row_0_sum)
print("All row sums:", attn_weights.sum(dim=-1))

1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)


### Compute All context vectors

In [29]:
all_context_vecs = attn_weights @ rounded_embeddings
print(all_context_vecs)
print("Previous 2nd context vector:", context_vec_2)

tensor([[0.5620, 0.6082, 0.3713],
        [0.5880, 0.6288, 0.3861],
        [0.6548, 0.7270, 0.4321],
        [0.5726, 0.6184, 0.3858],
        [0.5685, 0.6089, 0.3678],
        [0.6212, 0.7141, 0.4392]], grad_fn=<MmBackward0>)
Previous 2nd context vector: tensor([0.5880, 0.6288, 0.3861], grad_fn=<AddBackward0>)


## Self Attention

**Weight parameters** are learned coefficients that define the network connections, while **attention weights** are dynamic, context-specific values.

In [30]:
x_2 = rounded_embeddings[1] # second input element
d_in = rounded_embeddings.shape[1] #depends on the embeddings dimension the input embedding size, dim=3
d_out = 2 # the output embedding size, dim=2

torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

print(W_query)
print(W_key)
print(W_value)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])
Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]])
Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])


In [31]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

# calculate keys and values vector for all inputs
keys = rounded_embeddings @ W_key
values = rounded_embeddings @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

tensor([0.2857, 0.8093], grad_fn=<SqueezeBackward4>)
keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [32]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(0.4757, grad_fn=<DotBackward0>)


In [33]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([0.3565, 0.4757, 1.0404, 0.4996, 0.2961, 1.1539],
       grad_fn=<SqueezeBackward4>)


The difference to earlier is that we now scale the attention scores by dividing them by the square root of the embedding dimension,  𝑑𝑘‾‾‾√  (i.e., d_k**0.5):

Imagine you have two vectors, and their dot product results in a large value. When this large value is passed through the softmax function, it might dominate the probabilities, making the attention mechanism less sensitive to other relevant parts of the input. Scaling helps to mitigate this issue by preventing any single dot product from becoming overly influential.

In [34]:
d_k = keys.shape[1] # dimension of keys
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1328, 0.1444, 0.2153, 0.1469, 0.1272, 0.2333],
       grad_fn=<SoftmaxBackward0>)


In [35]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.2862, 0.6841], grad_fn=<SqueezeBackward4>)


### Compact SelfAttention Class

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- We can streamline the implementation above using PyTorch's Linear layers instead of torch random, which are equivalent to a matrix multiplication if we disable the bias units
- Another big advantage of using `nn.Linear` over our manual `nn.Parameter(torch.rand(...)` approach is that `nn.Linear` has a preferred weight initialization scheme, which leads to more stable model training

In [36]:
import torch.nn as nn

In [37]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

In [38]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(rounded_embeddings))

tensor([[-0.0125,  0.1567],
        [-0.0123,  0.1572],
        [-0.0129,  0.1581],
        [-0.0121,  0.1572],
        [-0.0126,  0.1566],
        [-0.0125,  0.1582]], grad_fn=<MmBackward0>)


### Hiding futer words with causal attention (one step back)

In [39]:
# Reuse data from previous section
queries = sa_v2.W_query(rounded_embeddings)
keys = sa_v2.W_key(rounded_embeddings)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1609, 0.1703, 0.1687, 0.1708, 0.1603, 0.1689],
        [0.1583, 0.1731, 0.1694, 0.1732, 0.1579, 0.1681],
        [0.1494, 0.1786, 0.1725, 0.1796, 0.1481, 0.1718],
        [0.1591, 0.1731, 0.1691, 0.1728, 0.1590, 0.1670],
        [0.1610, 0.1699, 0.1688, 0.1706, 0.1603, 0.1695],
        [0.1503, 0.1790, 0.1720, 0.1794, 0.1494, 0.1699]],
       grad_fn=<SoftmaxBackward0>)


Applying negative infinity effectively zeros out the probabilities for these future tokens in the subsequent softmax calculation.

In [40]:
context_length = attn_scores.shape[-1]
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[-1.5175e-04,        -inf,        -inf,        -inf,        -inf,
                -inf],
        [-6.5602e-03,  1.2047e-01,        -inf,        -inf,        -inf,
                -inf],
        [-5.4193e-03,  2.4702e-01,  1.9771e-01,        -inf,        -inf,
                -inf],
        [-9.1383e-03,  1.1044e-01,  7.7196e-02,  1.0829e-01,        -inf,
                -inf],
        [ 2.0232e-03,  7.7698e-02,  6.8403e-02,  8.3507e-02, -4.7449e-03,
                -inf],
        [-1.0605e-02,  2.3689e-01,  1.8059e-01,  2.4008e-01, -1.8712e-02,
          1.6290e-01]], grad_fn=<MaskedFillBackward0>)


In [41]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4776, 0.5224, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2985, 0.3569, 0.3446, 0.0000, 0.0000, 0.0000],
        [0.2360, 0.2568, 0.2508, 0.2564, 0.0000, 0.0000],
        [0.1939, 0.2045, 0.2032, 0.2054, 0.1930, 0.0000],
        [0.1503, 0.1790, 0.1720, 0.1794, 0.1494, 0.1699]],
       grad_fn=<SoftmaxBackward0>)


### Masking additional attention weights with dropout

In [42]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))
print(dropout(attn_weights))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])
tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6893, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5136, 0.0000, 0.5128, 0.0000, 0.0000],
        [0.0000, 0.4091, 0.4064, 0.4108, 0.3859, 0.0000],
        [0.3005, 0.3580, 0.0000, 0.0000, 0.2988, 0.3398]],
       grad_fn=<MulBackward0>)


### Causal Attention Class

In [43]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        print(b)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

In [46]:
torch.manual_seed(123)

batch = torch.stack((rounded_embeddings, rounded_embeddings), dim=0)
print(batch.shape) # 2 batches with 6 tokens each, and each token has embedding dimension 3

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 6, 3])
2
tensor([[[-0.2810, -0.1618],
         [-0.3837, -0.1213],
         [-0.5832, -0.2144],
         [-0.5292, -0.1581],
         [-0.4817, -0.1670],
         [-0.5365, -0.1788]],

        [[-0.2810, -0.1618],
         [-0.3837, -0.1213],
         [-0.5832, -0.2144],
         [-0.5292, -0.1581],
         [-0.4817, -0.1670],
         [-0.5365, -0.1788]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


## Self Attention multi-head

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

In [48]:
# Multiple heads to extract differente type of information, every head is using different initialized weights
# cada head da como resultado context vectors de cierte dimension que al final son concatenados

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape #b is for batches
        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        print('--------------------')
        print(f'{keys.shape} vs {keys.transpose(1,2).shape}')
        print('--------------------')
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

--------------------
torch.Size([2, 6, 2, 1]) vs torch.Size([2, 2, 6, 1])
--------------------
tensor([[[0.2035, 0.5207],
         [0.2297, 0.4747],
         [0.2453, 0.3592],
         [0.2474, 0.3960],
         [0.2365, 0.4187],
         [0.2444, 0.3881]],

        [[0.2035, 0.5207],
         [0.2297, 0.4747],
         [0.2453, 0.3592],
         [0.2474, 0.3960],
         [0.2365, 0.4187],
         [0.2444, 0.3881]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
